Install  the  packages





In [1]:
!pip install --quiet openai langchain
!pip install --quiet -U lancedb
!pip install pypdf
!pip install sentence-transformers
!pip install unstructured
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222

### Data Loading

Download [data](https://github.com/akashAD98/llam2_with_lancedb_/tree/main/multiple_pdf) and upload it in your google drive to run this example.
While this step is in process,
Take a break ☕

Once this step is done, Now you are ready to move with next steps

Mount your drive for uploading data & model

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

create folder inside drive & navigate to it
while creating this example folder with name `llm` inside google drive is used.



In [ ]:
%cd /content/drive/llm

### Import Packages

In [ ]:
import lancedb
import re
import pickle
import requests
import zipfile
from pathlib import Path
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.vectorstores import LanceDB
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

Let's load the embedding model
The all-mpnet-base-v2 model provides the best quality, while all-MiniLM-L6-v2 is 5 times faster and still offers good quality. you can try different models
we are using all -MiniLM-L6-V2 models for this demo, & we are using cpu for inference

In [ ]:
# load embedding model

embeddings_mini = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"}
)

select anyone below method

In [ ]:
# Load the  multiple pdfs
pdf_folder_path = "/content/drive/llm/pdf_paths/"

from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [ ]:
# # load single pdf
# from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader("/content/pdfs/layoutparser.pdf")
# pages = loader.load_and_split()


 Now that we have our raw documents loaded, we need to pre-process them to generate embeddings:



 Initialize a RecursiveCharacterTextSplitter to split text documents into smaller chunks.
 This helps in processing large text data efficiently.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(docs)
embeddings = embeddings_mini

Lets use lanceDB as vectore store database & crete temp table

In [ ]:
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "pandas_docs",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)
docsearch = LanceDB.from_documents(documents, embeddings, connection=table)

Now, as we are using the Llama 2 quantized model, you need to download and upload it to your Google Drive. You can choose different versions based on your system specifications. For demonstration purposes, we are using a small model, but feel free to explore other options
download the models from Thebloke repo.


https://huggingface.co/TheBloke/Llama-2-7B-GGML/tree/main

download the .bin file & kept in your current directory

In [ ]:
# Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model="llama-2-7b-chat.ggmlv3.q2_K.bin",
        model_type="llama",
        max_new_tokens=512,
        temperature=0.6,
    )
    return llm


qa = RetrievalQA.from_chain_type(
    llm=load_llm(), chain_type="stuff", retriever=docsearch.as_retriever()
)

You can ask the quetions to your data & i will give you response

In [ ]:
query = "What is DIA?"
qa.run(query)

In [ ]:
query = "What is LayoutParser ?"
qa.run(query)